# Twitter User Details in Python

This notebook uses the Tweepy library for Python to access Twitter's streaming API to get tweets in real-time. The notebook demonstrates how to access user details for an individual account or list of accounts.

In order to use the Twitter Streaming API you will need to [Apply for a Twitter Developer Account](https://developer.twitter.com/en/application/use-case). You will then need to set up a Twitter appand obtain the authentication credentials that will enable your application to connect to the API. See [Twitter developer apps: Overview](https://developer.twitter.com/en/docs/basics/apps/overview) for further details.

#### Import Dependencies

In [1]:
# Import Tweepy library to connect to the Twitter API and process the response
import tweepy
# Import Pandas to work with dataframes
import pandas as pd
# Import wget to retrieve profile images
import wget
# Import OS to manipulate file paths and names
import os
# Import Time to handle API rate limiting
import time

#### Authentication Details

In [2]:
# Enter the details from the Twitter App you created here
consumer_key = ''
consumer_secret = ''
access_token = ''
access_token_secret = ''

#### Authentication

In [3]:
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth) 

## Get User's Profile Details

The `get_user()` method accepts the user's ID or screen name.

#### Specify user ID

In [4]:
# Using screen name
_id = 'virt_arch'

#### Call the Twitter API

In [5]:
user = api.get_user(_id)

#### Load response into a dataframe

In [6]:
df = pd.DataFrame({'user_id': [user.id_str],
                   'screen_name': [user.screen_name],
                   'location': [user.location],
                   'profile_image': [user.profile_image_url],
                   'description': [user.description], 
                   'followers': [user.followers_count], 
                   'friends': [user.friends_count],
                   'created_at': [user.created_at], 
                   'verified': [user.verified]
                  })

#### Display the dataframe

In [7]:
df

,user_id,screen_name,location,profile_image,description,followers,friends,created_at,verified
0,2247929742,Virt_Arch,,http://pbs.twimg.com/profile_images/9732301081...,Research Assistant in Connected Environments @...,883,1715,2013-12-16 00:16:23,False


#### Download and save the user's profile picture

In [ ]:
# Remove the '_normal' tag from the URL to get a full sized image
link = user.profile_image_url.replace('_normal','')

# Set the ouput location for the image and save as png
destination = os.path.join('logos/' + user.screen_name + '.png')

# Use wget to download and save the image file
wget.download(link, destination)

#### Use HERE Maps API to geocode the user's location

In [ ]:
import herepy

geocoderApi = herepy.GeocoderApi('')

response = geocoderApi.free_form(user.location)

#### Display the returned coordinates

In [ ]:
print(response.items[0]['position'])

#### Append coordinates to the user dataframe

In [ ]:
df['lon'] = [response.items[0]['position']['lng']]
df['lat'] = [response.items[0]['position']['lat']]

#### Display the dataframe

In [ ]:
df

## Get User's Followers

The `followers()` method accepts the user's ID or screen name. We will use the `Cursor()` method to iterate through items. The number of items can be specific in `items()` or else left blank to iteract through all items. 

This request is subject to [rate limiting](https://developer.twitter.com/ja/docs/basics/rate-limits) against the 'Users' endpoint. It is recommended to set the rate limit of several seconds to avoid exceeding that rate limit.

#### Set time for rate limiter (seconds)

In [6]:
rate_limit = 5

#### Specify User

In [7]:
# Using screen name
_id = 'virt_arch'

#### Create Empty Dataframe

In [8]:
df = pd.DataFrame(columns=['user_id',
                           'screen_name',
                           'location',
                           'profile_image',
                           'description',
                           'followers',
                           'friends',
                           'created_at',
                           'verified'
                          ])

#### Call the Twitter API

In [11]:
for user in tweepy.Cursor(api.followers, _id).items():
    temp_df = pd.DataFrame({'user_id': [user.id_str],
                            'screen_name': [user.screen_name],
                            'location': [user.location],
                            'profile_image': [user.profile_image_url],
                            'description': [user.description], 
                            'followers': [user.followers_count], 
                            'friends': [user.friends_count],
                            'created_at': [user.created_at], 
                            'verified': [user.verified]
                           })
    df = df.append(temp_df)
    
    try:
        # Remove the '_normal' tag from the URL to get a full sized image
        link = user.profile_image_url.replace('_normal','')

        # Set the ouput location for the image and save as png
        destination = os.path.join('logos/' + user.screen_name + '.png')

        # Use wget to download and save the image file
        wget.download(link, destination)
    except:
        print(user.screen_name + ' profile image not available')
    
    print(df)
    time.sleep(rate_limit)

               user_id      screen_name                       location  \
0           1886108436      TwoGoatsInc                   New York, NY   
0             36388601         RoblemVR                 Greater Boston   
0           3648378016           hauniv                                  
0            296234919          OCliona                                  
0            306233350   TheSaintOfTous  Not a place, but a direction.   
0  1116262622431674368          mattnkm        Sydney, New South Wales   
0           4332391786      _annie_ring                London, England   
0           1393104673   woman_of_ideas       Where in the world am I?   
0            484895837      lab_normals                                  
0           2356057250     v830108ku543                 United Kingdom   
0   767057081891491840  creativecodedub           Dublin City, Ireland   
0   986515174206124032        see_serve                                  
0  1049391908852781057  UsamaAb3760030

               user_id      screen_name                       location  \
0           1886108436      TwoGoatsInc                   New York, NY   
0             36388601         RoblemVR                 Greater Boston   
0           3648378016           hauniv                                  
0            296234919          OCliona                                  
0            306233350   TheSaintOfTous  Not a place, but a direction.   
0  1116262622431674368          mattnkm        Sydney, New South Wales   
0           4332391786      _annie_ring                London, England   
0           1393104673   woman_of_ideas       Where in the world am I?   
0            484895837      lab_normals                                  
0           2356057250     v830108ku543                 United Kingdom   
0   767057081891491840  creativecodedub           Dublin City, Ireland   
0   986515174206124032        see_serve                                  
0  1049391908852781057  UsamaAb3760030

               user_id      screen_name                       location  \
0           1886108436      TwoGoatsInc                   New York, NY   
0             36388601         RoblemVR                 Greater Boston   
0           3648378016           hauniv                                  
0            296234919          OCliona                                  
0            306233350   TheSaintOfTous  Not a place, but a direction.   
0  1116262622431674368          mattnkm        Sydney, New South Wales   
0           4332391786      _annie_ring                London, England   
0           1393104673   woman_of_ideas       Where in the world am I?   
0            484895837      lab_normals                                  
0           2356057250     v830108ku543                 United Kingdom   
0   767057081891491840  creativecodedub           Dublin City, Ireland   
0   986515174206124032        see_serve                                  
0  1049391908852781057  UsamaAb3760030

               user_id      screen_name                       location  \
0           1886108436      TwoGoatsInc                   New York, NY   
0             36388601         RoblemVR                 Greater Boston   
0           3648378016           hauniv                                  
0            296234919          OCliona                                  
0            306233350   TheSaintOfTous  Not a place, but a direction.   
0  1116262622431674368          mattnkm        Sydney, New South Wales   
0           4332391786      _annie_ring                London, England   
0           1393104673   woman_of_ideas       Where in the world am I?   
0            484895837      lab_normals                                  
0           2356057250     v830108ku543                 United Kingdom   
0   767057081891491840  creativecodedub           Dublin City, Ireland   
0   986515174206124032        see_serve                                  
0  1049391908852781057  UsamaAb3760030

               user_id      screen_name                       location  \
0           1886108436      TwoGoatsInc                   New York, NY   
0             36388601         RoblemVR                 Greater Boston   
0           3648378016           hauniv                                  
0            296234919          OCliona                                  
0            306233350   TheSaintOfTous  Not a place, but a direction.   
0  1116262622431674368          mattnkm        Sydney, New South Wales   
0           4332391786      _annie_ring                London, England   
0           1393104673   woman_of_ideas       Where in the world am I?   
0            484895837      lab_normals                                  
0           2356057250     v830108ku543                 United Kingdom   
0   767057081891491840  creativecodedub           Dublin City, Ireland   
0   986515174206124032        see_serve                                  
0  1049391908852781057  UsamaAb3760030

...

               user_id     screen_name                       location  \
0           1886108436     TwoGoatsInc                   New York, NY   
0             36388601        RoblemVR                 Greater Boston   
0           3648378016          hauniv                                  
0            296234919         OCliona                                  
0            306233350  TheSaintOfTous  Not a place, but a direction.   
..                 ...             ...                            ...   
0           2966225919       AVRLondon            Shad Thames, London   
0   999945589260537856      NThylstrup                        Denmark   
0   867312279355871234   NoortjeMarres                                  
0           2531516672      NerdButler                Dallas, TX, USA   
0            279045646  martinkonyango                        Nairobi   

                                        profile_image  \
0   http://pbs.twimg.com/profile_images/1145698004...   
0   http:

       user_id     screen_name                       location  \
0   1886108436     TwoGoatsInc                   New York, NY   
0     36388601        RoblemVR                 Greater Boston   
0   3648378016          hauniv                                  
0    296234919         OCliona                                  
0    306233350  TheSaintOfTous  Not a place, but a direction.   
..         ...             ...                            ...   
0   2531516672      NerdButler                Dallas, TX, USA   
0    279045646  martinkonyango                        Nairobi   
0    117046680       aidankane                Galway, Ireland   
0   3257361870       vishnuZsf                                  
0    732376351       yuhanji16                         London   

                                        profile_image  \
0   http://pbs.twimg.com/profile_images/1145698004...   
0   http://pbs.twimg.com/profile_images/1094301771...   
0   http://pbs.twimg.com/profile_images/12645345

                user_id      screen_name                       location  \
0            1886108436      TwoGoatsInc                   New York, NY   
0              36388601         RoblemVR                 Greater Boston   
0            3648378016           hauniv                                  
0             296234919          OCliona                                  
0             306233350   TheSaintOfTous  Not a place, but a direction.   
..                  ...              ...                            ...   
0            3257361870        vishnuZsf                                  
0             732376351        yuhanji16                         London   
0             990725857          con20or                        Ireland   
0   1013751579634290688  OASArchitecture                London, England   
0            1582637953    GsureInsights       Brighton, United Kingdom   

                                        profile_image  \
0   http://pbs.twimg.com/profile_images/11

                user_id      screen_name                       location  \
0            1886108436      TwoGoatsInc                   New York, NY   
0              36388601         RoblemVR                 Greater Boston   
0            3648378016           hauniv                                  
0             296234919          OCliona                                  
0             306233350   TheSaintOfTous  Not a place, but a direction.   
..                  ...              ...                            ...   
0   1013751579634290688  OASArchitecture                London, England   
0            1582637953    GsureInsights       Brighton, United Kingdom   
0            3169843239          Tridify                                  
0            3410891447  johan_hanegraaf        Rotterdam, Zuid-Holland   
0            1140765066  IrishTechEvents                        Ireland   

                                        profile_image  \
0   http://pbs.twimg.com/profile_images/11

KeyboardInterrupt: 

#### Display Dataframe

In [12]:
df

,user_id,screen_name,location,profile_image,description,followers,friends,created_at,verified
0,1886108436,TwoGoatsInc,"New York, NY",http://pbs.twimg.com/profile_images/1145698004...,Award-Winning Creative Technology Studio focus...,10821,7294,2013-09-20 10:47:29,False
0,36388601,RoblemVR,Greater Boston,http://pbs.twimg.com/profile_images/1094301771...,#Futurist #ThoughtLeader #Tech #Influencer #So...,51942,26978,2009-04-29 15:44:19,False
0,3648378016,hauniv,,http://pbs.twimg.com/profile_images/1264534583...,Get monthly email newsletter of the most promi...,8632,2758,2015-09-14 02:17:59,False
0,296234919,OCliona,,http://pbs.twimg.com/profile_images/1101443744...,,108,1064,2011-05-10 12:29:05,False
0,306233350,TheSaintOfTous,"Not a place, but a direction.",http://pbs.twimg.com/profile_images/1093160059...,"Writer, Cultural Omnivore // Bylines ✍🏾 @ Past...",1375,1236,2011-05-27 14:35:36,False
...,...,...,...,...,...,...,...,...,...
0,3169843239,Tridify,,http://pbs.twimg.com/profile_images/9611599021...,Publish your BIM straight to 3D models on the ...,1709,2537,2015-04-15 12:16:29,False
0,3410891447,johan_hanegraaf,"Rotterdam, Zuid-Holland",http://pbs.twimg.com/profile_images/1053696094...,"architect, developer, design technology, BIM, ...",1046,501,2015-08-09 18:38:57,False
0,1140765066,IrishTechEvents,Ireland,http://pbs.twimg.com/profile_images/1101100242...,Listings of All Tech Meetups and Courses in Ir...,3567,2693,2013-02-01 21:34:03,False
0,3300885028,data4change,"London, England",http://pbs.twimg.com/profile_images/1104047496...,We help Civil Society Organisations to create ...,2484,1653,2015-05-27 20:03:32,False


#### Save Dataframe to CSV

In [13]:
df.to_csv(os.path.join('data/' + _id + '_' + 'followers.csv'))